## Cài đặt môi trường

In [2]:
!pip install transformers torch accelerate
!pip install langchain_openai langchain_community
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
!pip install gdown -U --no-cache-dir

  Attempting uninstall: gdown
    Found existing installation: gdown 5.1.0
    Uninstalling gdown-5.1.0:
      Successfully uninstalled gdown-5.1.0


In [4]:

import gdown
url = "https://drive.google.com/drive/folders/1bIiWoKNJudkUUTce8qIU6fNUbWWU1SOD?usp=sharing"
gdown.download_folder(url)

Retrieving folder contents


Processing file 12KoIvxI__JUH_aGpjFCduUAJld-xmEXn ĐIỂM RÈN LUYỆN.txt
Processing file 1Wm6tXKgTZcnGkDdCyjUHs5Y_85AkkP0W QUY CHẾ ĐÀO TẠO.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=12KoIvxI__JUH_aGpjFCduUAJld-xmEXn
To: /content/DATA/ĐIỂM RÈN LUYỆN.txt
100%|██████████| 22.2k/22.2k [00:00<00:00, 22.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Wm6tXKgTZcnGkDdCyjUHs5Y_85AkkP0W
To: /content/DATA/QUY CHẾ ĐÀO TẠO.txt
100%|██████████| 54.4k/54.4k [00:00<00:00, 56.7MB/s]
Download completed


['/content/DATA/ĐIỂM RÈN LUYỆN.txt', '/content/DATA/QUY CHẾ ĐÀO TẠO.txt']

In [5]:
import os
os.environ['OPENAI_API_KEY'] = "from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import TextSplitter
from langchain_community.vectorstores import FAISS
class LineTextSplitter(TextSplitter):
    def split_text(self, text):
        return text.split('\n')
text_splitter = LineTextSplitter()

## Xử lý dữ liệu

### Điểm rèn luyện

In [6]:

with open('/content/DATA/ĐIỂM RÈN LUYỆN.txt', 'r', encoding='utf-8') as f:
    text = f.read()
documents = text_splitter.split_text(text)
db = FAISS.from_texts(documents, OpenAIEmbeddings(model="text-embedding-3-large"))
db.save_local("/content/index/DRL_index")

### Quy chế đào tạo


In [7]:
with open('/content/DATA/QUY CHẾ ĐÀO TẠO.txt', 'r', encoding='utf-8') as f:
    text = f.read()
documents = text_splitter.split_text(text)
db = FAISS.from_texts(documents, OpenAIEmbeddings(model="text-embedding-3-large"))
db.save_local("/content/index/QCDT_index")

## Định nghĩa các hàm truy vấn thông tin

In [8]:
def DRL(info):

    db = FAISS.load_local("index/DRL_index", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization = True)
    results = db.similarity_search(info, k=5)
    docs = [{"content": doc.page_content} for doc in results]
    docs_string = json.dumps(docs, ensure_ascii=False)

    return docs_string


def QCDT(info):

    db = FAISS.load_local("index/QCDT_index", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization = True)
    results = db.similarity_search(info, k=5)
    docs = [{"content": doc.page_content} for doc in results]
    docs_string = json.dumps(docs, ensure_ascii=False)

    return docs_string

## Định nghĩa metadata cho các hàm

In [9]:
import json

functions_metadata = [
{
        "type": "function",
        "function": {
            "name": "DRL",
            "description": "Cung cấp những thông tin liên quan đến điểm rèn luyện mà bạn cần biết.",
            "parameters": {
                "type": "object",
                "properties": {
                    "info": {
                        "type": "string",
                        "description": "Thông tin mà bạn cần tìm kiếm, e.g. Chấp hành và không vi phạm: +15đ rèn luyện ",
                    },
                },
                "required": ["info"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "QCDT",
            "description": "Cung cấp những thông tin liên quan đến quy chế đào tạo mà bạn cần biết.",
            "parameters": {
                "type": "object",
                "properties": {
                    "info": {
                        "type": "string",
                        "description": "Thông tin mà bạn cần tìm kiếm, e.g. Trường đại học Khoa học Tự nhiên có 2 cơ sở.",
                    },
                },
                "required": ["info"],
            },
        },
    },
]

## Khởi tạo bộ nhớ cho chatbot

In [10]:
messages = [
    {"role": "system", "content": f"""Bạn là một trợ lý hữu ích của trường đại học Khoa học Tự nhiên, có quyền truy cập vào các chức năng sau, sử dụng chúng nếu cần -\n{str(functions_metadata)} Để sử dụng các chức năng này, hãy phản hồi với:\n<functioncall> {{\\"name\\": \\"function_name\\", \\"arguments\\": {{\\"arg_1\\": \\"value_1\\", \\"arg_1\\": \\"value_1\\", ...}} }} </functioncall>\n\nTrường hợp đặc biệt bạn phải xử lý:\n - Nếu không có chức năng nào khớp với yêu cầu của người dùng, bạn sẽ phản hồi một cách lịch sự rằng bạn không thể giúp được.""" },
]

## Tải model và tokenizer từ Hugging Face

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained('hiieu/Vistral-7B-Chat-function-calling')
model = AutoModelForCausalLM.from_pretrained(
    'hiieu/Vistral-7B-Chat-function-calling',
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

## Định nghĩa các hàm hỗ trợ

In [12]:
def remove_extra_quotes(function_call_string):

  start = function_call_string.find('{')
  end = function_call_string.rfind('}')
  if start != -1 and end != -1:
    json_string = function_call_string[start:end+1]
    try:
      json.loads(json_string)
      return function_call_string
    except json.JSONDecodeError:
      json_string = json_string.replace("'{", "{").replace("'}", "}")
      try:
        json.loads(json_string)
        return function_call_string[:start] + json_string + function_call_string[end+1:]
      except json.JSONDecodeError:
        return None
  else:
    return None

def remove_simulated_conversation(text):
  index = text.find(".user")
  if index != -1:
    return text[:index + 1]
  else:
    return text

## Hàm chính để sử dụng mô hình dưới dạng chatbot

In [1]:
import re
def ask_AI(query):
  # Thêm câu hỏi của người dùng vào bộ nhớ
  messages = [
    {"role": "system", "content": f"""Bạn là một trợ lý hữu ích của trường đại học Khoa học Tự nhiên, có quyền truy cập vào các chức năng sau, sử dụng chúng nếu cần -\n{str(functions_metadata)} Để sử dụng các chức năng này, hãy phản hồi với:\n<functioncall> {{\\"name\\": \\"function_name\\", \\"arguments\\": {{\\"arg_1\\": \\"value_1\\", \\"arg_1\\": \\"value_1\\", ...}} }} </functioncall>\n\nTrường hợp đặc biệt bạn phải xử lý:\n - Nếu không có chức năng nào khớp với yêu cầu của người dùng, bạn sẽ phản hồi một cách lịch sự rằng bạn không thể giúp được.""" },
]
  messages.append({"role": "user", "content": query})
  # Mã hóa văn bản thành input ids cho model
  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

  # Sinh text từ model
  outputs = model.generate(
    input_ids,
    max_new_tokens=768,
    do_sample=True,
    top_p=0.95,
    top_k=40,
    temperature=0.1,
    repetition_penalty=1.05,
  )

  response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  match = re.search(r"<functioncall>.*</functioncall>", response)

  if match:
    function_call_string = match.group(0)
    messages.append({"role": "assistant", "content": function_call_string})
    json_match = re.search(r"{.*}", function_call_string)
    if json_match:
      json_string = json_match.group(0)
      json_string = remove_extra_quotes(json_string)
      try:
        function_call = json.loads(json_string)
        function_name = function_call["name"]
        function_args = function_call["arguments"]
        if function_name == "DRL":
          info = function_args["info"]
          fnc_result = DRL(info)
          messages.append({"role": "user", "content": f"""<function_response> {fnc_result} </function_response>"""})
        elif function_name == "QCDT":
          info = function_args["info"]
          fnc_result = QCDT(info)
          messages.append({"role": "user", "content": f"""<function_response> {fnc_result} </function_response>"""})
        else:
          print("Unknown function called.")
        input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
).to(model.device)

        outputs = model.generate(
        input_ids,
        max_new_tokens=768,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        temperature=0.1,
        repetition_penalty=1.05,
        )

        response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        return response

      except json.JSONDecodeError:
        print("Invalid function call format.")

  else:
    messages.append({"role": "assistant", "content": response})
    return response

## Sử dụng chatbot

In [13]:
res = ask_AI("xin chào")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [14]:
res

' Xin chào! Tôi có thể hỗ trợ gì cho bạn hôm nay? '

In [17]:
res = ask_AI("Tham gia câu lạc bộ có được cộng điểm rèn luyện không")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
res

' Tham gia câu lạc bộ có thể góp phần vào việc đạt được điểm rèn luyện vì nó thuộc nhóm "2. Ý thức và thái độ tham gia các câu lạc bộ học thuật, các hoạt động học thuật, " theo Điều 10. Cách tính điểm rèn luyện. Tuy nhiên, số điểm cụ thể phụ thuộc vào nhiều yếu tố khác nhau như mức độ tham gia và chất lượng của câu lạc bộ. '